In [1]:
!pip install albumentations 
!pip install kerascv mxnet 

    100% |████████████████████████████████| 61kB 4.0MB/s 
    100% |████████████████████████████████| 634kB 24.6MB/s 
  Stored in directory: /tmp/.cache/pip/wheels/92/8d/52/c0c6d4d3f42caafcc27b3b3c5cfe990e59635fe254349c60aa
  Stored in directory: /tmp/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built albumentations imgaug
  Found existing installation: imgaug 0.2.8
    Uninstalling imgaug-0.2.8:
      Successfully uninstalled imgaug-0.2.8
    100% |████████████████████████████████| 92kB 5.3MB/s 
    100% |████████████████████████████████| 28.4MB 1.4MB/s 
    100% |████████████████████████████████| 13.8MB 2.8MB/s 
tensorpack 0.9.1 has requirement msgpack-numpy>=0.4.4.2, but you'll have msgpack-numpy 0.4.3.2 which is incompatible.
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
kmodes 0.9 has requirement scikit-learn<0.20.0,>=0.19.0, but you'll have scikit-learn 0.20.3 which is incompatible.
km

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from glob import glob
from random import shuffle
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution1D, concatenate, SpatialDropout1D, GlobalMaxPool1D, GlobalAvgPool1D, Embedding, \
    Conv2D, SeparableConv1D, Add, BatchNormalization, Activation, GlobalAveragePooling2D, LeakyReLU, Flatten
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, \
    Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D
from keras.layers.pooling import _GlobalPooling1D
from keras.losses import mae, sparse_categorical_crossentropy, binary_crossentropy
from keras.models import Model
from keras.applications.nasnet import NASNetMobile, NASNetLarge, preprocess_input
from keras.optimizers import Adam, RMSprop
from keras.callbacks import *
from imgaug import augmenters as iaa
from albumentations import Compose, RandomRotate90, Transpose, Flip, OneOf, CLAHE, IAASharpen, IAAEmboss, RandomBrightnessContrast, JpegCompression, Blur, GaussNoise, HueSaturationValue, ShiftScaleRotate, Normalize
import imgaug as ia
#from shufflenetv2 import ShuffleNetV2
from kerascv.model_provider import get_model as kecv_get_model

from keras.regularizers import l2

print(os.listdir("../input"))

import shutil,math

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['deepshuffle', 'fdmobilenet', 'histopathologic-cancer-detection']


In [3]:
img_size_in = 224

In [4]:
df_train = pd.read_csv("../input/histopathologic-cancer-detection/train_labels.csv")
id_label_map = {k:v for k,v in zip(df_train.id.values, df_train.label.values)}
df_train.head()

,id,label
0,f38a6374c348f90b587e046aac6079959adf3835,0
1,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
2,755db6279dae599ebb4d39a9123cce439965282d,0
3,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
4,068aba587a4950175d04c680d38943fd488d6a9d,0


In [5]:
def get_id_from_file_path(file_path):
    return file_path.split(os.path.sep)[-1].replace('.tif', '')

In [6]:
labeled_files = glob('../input/histopathologic-cancer-detection/train/*.tif')
test_files = glob('../input/histopathologic-cancer-detection/test/*.tif')

In [7]:
print("labeled_files size :", len(labeled_files))
print("test_files size :", len(test_files))

labeled_files size : 220025
test_files size : 57458


In [8]:
train, val = train_test_split(labeled_files, test_size=0.1, random_state=101010)

In [9]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def do_train_augmentations():
    return Compose([
        RandomRotate90(p=0.5),
        Transpose(p=0.5),
        Flip(p=0.5),
        OneOf([CLAHE(clip_limit=2),
              IAASharpen(),
              IAAEmboss(),
              RandomBrightnessContrast(),
              JpegCompression(),
              Blur(),
              GaussNoise()],
              p=0.5),
        HueSaturationValue(p=0.5),
        ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=45, p=0.5),
        Normalize(p=1)])


def do_inference_aug():
    return Compose([Normalize(p=1)], p=1)

def data_gen(list_files, id_label_map_in, batch_size_in, img_size_in, aug_funtion):
    aug = aug_funtion()
    
    while True:
        shuffle(list_files)
        for block in chunker(list_files, batch_size_in):

            X = [cv2.resize(cv2.imread(x), (img_size_in, img_size_in)) for x in block]
            X = [aug(image=x)['image'] for x in X]

            Y = [id_label_map_in[get_id_from_file_path(x)] for x in block]

            yield np.array(X), np.array(Y)
    

In [10]:
def create_model(input_shape, n_out):
    inp_mask = Input(shape=input_shape)
    net = kecv_get_model("fdmobilenet_w3d4", pretrained=True)
    net.name = 'shufflenet'

    x = net(inp_mask)
    #x = net.output
    #x = Flatten()(x)
    x = Dense(512, activation="relu")(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = Dropout(0.25)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.25)(x)
    out = Dense(n_out, activation="sigmoid")(x)
    model = Model(inputs=inp_mask, outputs=[out])
    model.summary()    
    return model

In [11]:
model = create_model(input_shape=(img_size_in, img_size_in, 3), n_out=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
shufflenet (Model)           (None, 1000)              1845112   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               512512    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [12]:
import shutil,math
#-----In case you want to use a learning rate scheduler from keras this is a good step decay function to play around with-----#
def step_decay(epoch):
    initial_lrate=1e-4
    drop=0.6
    epochs_drop = 3.0
    lrate= initial_lrate * math.pow(drop,math.floor((1+epoch)/epochs_drop))
    return lrate

In [13]:
#-------Callbacks-------------#
h5_path = "model.h5"
checkpoint = ModelCheckpoint(h5_path, 
                             monitor='val_acc', 
                             verbose=1, 
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max')

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode='auto'
)

lrsched = LearningRateScheduler(step_decay,verbose=1)

callbacks = [checkpoint,lrsched]

In [14]:
batch_size=32

#h5_path = "model.h5"
#checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True,save_weights_only=True,mode='max')


for layer in model.layers:
    layer.trainable = False
    
model.layers[-1].trainable = True
model.layers[-2].trainable = True
model.layers[-3].trainable = True
model.layers[-4].trainable = True
model.layers[-5].trainable = True
model.layers[-6].trainable = True
model.layers[-7].trainable = True


model.compile(
    loss='binary_crossentropy', 
    optimizer=Adam(lr=1e-3),
    metrics=['acc'])

history = model.fit_generator(
    data_gen(train, id_label_map, batch_size,  img_size_in, do_train_augmentations),
    validation_data=data_gen(val, id_label_map, batch_size,img_size_in,do_inference_aug),
    epochs=2, verbose=1,
    #callbacks=[checkpoint],
    steps_per_epoch=len(train) // batch_size,
    validation_steps=len(val) // batch_size)


Epoch 1/2
6188/6188 [==============================] - 1319s 213ms/step - loss: 0.2959 - acc: 0.8813 - val_loss: 0.2667 - val_acc: 0.9084
Epoch 2/2
4687/6188 [=====================>........] - ETA: 4:22 - loss: 0.2431 - acc: 0.9057

In [15]:


# train all layers
for layer in model.layers:
    layer.trainable = True
    
model.load_weights("../input/fdmobilenet/FD-MobileNet_model.h5") 
    
model.compile(loss='binary_crossentropy',
            optimizer=Adam(lr=1e-4),
            metrics=['accuracy'])

batch_size=32


history = model.fit_generator(
    data_gen(train, id_label_map, batch_size,  img_size_in, do_train_augmentations),
    validation_data=data_gen(val, id_label_map, batch_size,img_size_in,do_inference_aug),
    epochs=20, verbose=1,
    callbacks=callbacks,
    steps_per_epoch=len(train) // batch_size,
    validation_steps=len(val) // batch_size)



Epoch 1/20

Epoch 00001: LearningRateScheduler setting learning rate to 0.0001.
6188/6188 [==============================] - 1157s 187ms/step - loss: 0.1605 - acc: 0.9412 - val_loss: 0.1248 - val_acc: 0.9576

Epoch 00001: val_acc improved from -inf to 0.95756, saving model to model.h5
Epoch 2/20

Epoch 00002: LearningRateScheduler setting learning rate to 0.0001.
6188/6188 [==============================] - 1128s 182ms/step - loss: 0.1579 - acc: 0.9423 - val_loss: 0.1182 - val_acc: 0.9589

Epoch 00002: val_acc improved from 0.95756 to 0.95885, saving model to model.h5
Epoch 3/20

Epoch 00003: LearningRateScheduler setting learning rate to 6e-05.
1608/6188 [======>.......................] - ETA: 13:08 - loss: 0.1548 - acc: 0.9433

In [16]:
#show_final_history(history)
#print("Validation Accuracy: " + str(history.history['val_acc'][-1:]))

In [17]:
#predictions = model.predict_generator(roc_validation_generator, steps=len(roc_validation_generator), verbose=1)
#false_positive_rate, true_positive_rate, threshold = roc_curve(roc_validation_generator.classes, predictions)
#area_under_curve = auc(false_positive_rate, true_positive_rate)

#plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(false_positive_rate, true_positive_rate, label='AUC = {:.3f}'.format(area_under_curve))
#plt.xlabel('False positive rate')
#plt.ylabel('True positive rate')
#plt.title('ROC curve')
#plt.legend(loc='best')
#plt.savefig(ROC_PLOT_FILE, bbox_inches='tight')
#plt.show()

In [18]:
preds = []
ids = []

In [19]:
for batch in chunker(test_files, batch_size):
    X = [preprocess_input(cv2.resize(cv2.imread(x),(img_size_in,img_size_in))) for x in batch]
    ids_batch = [get_id_from_file_path(x) for x in batch]
    X = np.array(X)
    preds_batch = ((model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
    preds += preds_batch
    ids += ids_batch

In [20]:
df = pd.DataFrame({'id':ids, 'label':preds})
df.to_csv("baseline_FD-MobileNet.csv", index=False)
df.head()

,id,label
0,d3a0e75dd14a12773d7ad53bda6f1e1c5ba97f5c,0.885244
1,bdb69de941bb1dedf3d15564b39a67dec276f701,0.000919
2,371816c763c118a62ac1f4139f45806167c7e88b,0.011555
3,d18e5df26368164b4cd531941e489f2f19a5302d,0.680979
4,d57c22c04cd9c20540edff394de0f50fcdf55d0d,0.992982
